# 1️⃣. Introducción y objetivo del análisis

## .1. Información de la base de datos

La base de datos es ficticia sobre las ventas de un supermercado tomada de <a href="https://www.kaggle.com/datasets/chadwambles/supermarket-sales?resource=download">Kaggle</a> 

El creador indica que la base incluye lo siguiente:
* ```sale_id```: ID de ventas único para cada fila.
* ```branch```: Sucursal del supermercado (New York, Chicago y Los Angeles).
* ```city```: Ciudad del supermercado (New York, Chicago y Los Angeles).
* ```customer_type```: Tipo de cliente (Member o Normal). Los miembros reciben puntos de recompensa.
* ```gender```: Género (Male o Female).
* ```product_name```: Nombre del producto vendido.
* ```product_category```: Categoría del producto vendido.
* ```unit_price```: Precio unitario de cada producto vendido.
* ```quantity```: Cantidad del producto vendido.
* ```tax```: 7% de impuesto sobre las ventas de cada producto.
* ```total_price```: Precio total del producto después de impuestos.
* ```reward_points```: Puntos de recompensa solo para miembros.

La moneda se supondrá en dólares ($)

## .2. Objetivo

Analizar ventas y comportamiento de clientes en un supermercado ficticio para obtener insights de negocio.

## .3. Preguntas de investigación

1. Ventas y rentabilidad
    * ¿Qué ciudad genera mayores ingresos totales?

    * ¿Qué día de la semana tiene mayor facturación?

    * ¿Cuál es el ticket promedio de compra?

2. Clientes
    * ¿Los miembros gastan más que los clientes normales?

    * ¿Hay diferencia en gasto por género?

3. Productos
    * ¿Qué categoría aporta más ingresos?

    * ¿Qué productos se venden en mayor cantidad vs. los que generan más dinero?

4. KPIs prácticos
    * Ventas totales 💰

    * Ticket promedio 🧾

    * Número de clientes únicos 👥

    * Top 5 productos más vendidos 📦

# 2️⃣. Carga y exploración de datos

## .1. Carga

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_ventas = pd.read_csv('archive/sales.csv', index_col='sale_id')
df_ventas

,branch,city,customer_type,gender,product_name,product_category,unit_price,quantity,tax,total_price,reward_points
sale_id,,,,,,,,,,,
1,A,New York,Member,Male,Shampoo,Personal Care,5.50,3,1.16,17.66,1
2,B,Los Angeles,Normal,Female,Notebook,Stationery,2.75,10,1.93,29.43,0
3,A,New York,Member,Female,Apple,Fruits,1.20,15,1.26,19.26,1
4,A,Chicago,Normal,Male,Detergent,Household,7.80,5,2.73,41.73,0
5,B,Los Angeles,Member,Female,Orange Juice,Beverages,3.50,7,1.72,26.22,2
...,...,...,...,...,...,...,...,...,...,...,...
996,A,New York,Member,Female,Shampoo,Stationery,1.55,11,1.19,18.24,1
997,A,New York,Member,Male,Detergent,Personal Care,2.44,7,1.20,18.28,1
998,A,New York,Member,Female,Shampoo,Stationery,17.92,2,2.51,38.35,3


## .2. Exploración

In [3]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   branch            1000 non-null   object 
 1   city              1000 non-null   object 
 2   customer_type     1000 non-null   object 
 3   gender            1000 non-null   object 
 4   product_name      1000 non-null   object 
 5   product_category  1000 non-null   object 
 6   unit_price        1000 non-null   float64
 7   quantity          1000 non-null   int64  
 8   tax               1000 non-null   float64
 9   total_price       1000 non-null   float64
 10  reward_points     1000 non-null   int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 93.8+ KB


.info() muestra que no hay nulos en la base de datos.

Además obtenemos las 
* variables categóricas [branch, city, customer_type, gender, product_name, product_category]
* variables numéricas [unit_price, quantity, tax, total_price, reward_points]

In [4]:
df_ventas.describe()

,unit_price,quantity,tax,total_price,reward_points
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,10.836110,10.337000,7.758010,118.583900,6.057000
std,5.775924,6.029908,6.538066,99.936441,9.350464
min,1.020000,1.000000,0.080000,1.210000,0.000000
25%,5.867500,5.000000,2.510000,38.380000,0.000000
50%,10.615000,10.000000,5.870000,89.705000,0.000000
75%,15.882500,16.000000,11.522500,176.072500,10.000000
max,20.980000,20.000000,28.390000,433.990000,43.000000


In [5]:
df_ventas.describe(include='object')

,branch,city,customer_type,gender,product_name,product_category
count,1000,1000,1000,1000,1000,1000
unique,2,3,2,2,5,5
top,A,New York,Member,Male,Shampoo,Fruits
freq,674,344,516,528,224,209


Información que nos brinda .describe().

* La base contiene 1000 registros de ventas. 
* Los precios unitarios (unit_price) van de 1 a 21, la cantidad (quantity) de 1 a 20 y el precio total(total_price) promedio es de aproximadamente 118. 
* En los datos categóricos, la sucursal más frecuente es la A (674 transacciones) y la ciudad con más registros es Nueva York (344). 
* Existe un balance entre clientes Member y Normal, así como entre géneros

Además el describe categórico permite observar las categorías únicas presentes, y hay un detalle que llama la atención en "branch", ya que la documentación del dataset menciona 3 sucursales, y en los datos cargados solo se encuentran 2 categorías para branch. Esto sugiere que la sucursal C no tiene registros en esta muestra o que hubo un error en la generación de los datos.

In [6]:
print(df_ventas['branch'].unique())
df_ventas['branch'].value_counts()

['A' 'B']


branch
A    674
B    326
Name: count, dtype: int64

In [7]:
df_ventas.duplicated().sum()

np.int64(0)

No hay valores nulos ni duplicados. Lo cual es esperable en una base de datos ficticia.

# 3️⃣. Limpieza y transformación

## .1. Revisión de columna branch

Dado al hallazgo anterior en la columna "branch", se procede a revisar minusciosamente esta columna junto con "city", ya que se esperaba que existira una sucursal en cada ciudad.

Como el describe categórico mostró que city si posee las 3 categorías espradas y las ciudades son New York, Chicago, Los Angeles y se sabe que ninguna de ellas es una subciudad de otra, entonces se revisará cual letra se le ha asignado a cada ciudad en la sucursal.

In [8]:
df_ventas.groupby('city').count()

,branch,customer_type,gender,product_name,product_category,unit_price,quantity,tax,total_price,reward_points
city,,,,,,,,,,
Chicago,330,330,330,330,330,330,330,330,330,330
Los Angeles,326,326,326,326,326,326,326,326,326,326
New York,344,344,344,344,344,344,344,344,344,344


In [9]:
df_ventas['branch'].value_counts()

branch
A    674
B    326
Name: count, dtype: int64

En la agrupación anterior se puede observar que cada ciudad tiene asociada cierta cantidad de datos:

| City| Cantidad compras |
|---|---|
| Chicago | 330 |
| Los Angeles | 326 |
| New York | 344 |

Ahora observemos la tabla obtenida de branch al contar las 2 categorías marcadas

| branch | Cantidad compras |
|---|---|
| A |   674 |
| B |   326 |

Dado que la suma de Chicago (330) y New York (344) coincide exactamente con el valor de la categoría A (674), y que **Los Angeles** coincide con la **categoría B** (326), se concluye que hubo un error de codificación. La reasignación más coherente es:

* Chicago → A
* Los Angeles → B
* New York → C

Esta corrección se asume con base en la distribución de los datos, para poder continuar con un análisis más consistente.

Por lo que quedaría de la siguiente forma:

| branch | city | Cantidad compras |
|---|---|---|
| A | Chicago | 330 |
| B | Los Angeles | 326 |
| C | New York | 344 |

## .2. Transformación de columna branch

In [ ]:
df_ventas.branch = df_ventas.city.map({'Chicago': 'A', 'Los Angeles': 'B', 'New York': 'C'})

df_ventas.branch.value_counts()

branch
C    344
A    330
B    326
Name: count, dtype: int64

## .3. Revisión de columna total_price

In [ ]:
expected_total = df_ventas['unit_price'] * df_ventas['quantity'] * 1.07
(df_ventas['total_price'] - expected_total).describe()

count    1000.000000
mean        0.000198
std         0.002927
min        -0.004900
25%        -0.002400
50%         0.000200
75%         0.002800
max         0.005000
dtype: float64

En la verificación anterior se confirma que los valores de la columna ```total_price``` son consistentes con el cálculo esperado a partir de las columnas ```unit_price```, ```quantity``` y ```tax``` de la base de datos.

Además, la diferencia observada en la validación del cálculo de ```total_price``` se debe a imprecisiones de punto flotante al operar decimales en Python.

# 4️⃣. Análisis exploratorio

# 5️⃣. Visualizaciones

# 6️⃣. Conclusiones

Se notó en la carga y exploración de datos que la base en una categoría tiene una limitación, ya que se esperaba que la columna "branch" tuviera 3 categorías, pero se notó que solo posee 2. Por lo que limita el análisis de la tercer sucursal.